In [1]:
from deep_traffic_generation.tcvae import TCVAE
from deep_traffic_generation.core.datasets import TrafficDataset
from traffic.core import Traffic

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
from os import walk

In [2]:
dataset1 = TrafficDataset.from_file(
    ("../deep_traffic_generation/data/training_datasets/takeoffs_south_LFPO_07.pkl"),
    features=["track", "groundspeed", "altitude", "timedelta"],
    scaler=MinMaxScaler(feature_range=(-1,1)),
    # scaler=None,
    shape="image",
    info_params={"features": ["latitude", "longitude"], "index": -1},
)
dataset1_bis = TrafficDataset.from_file(
    ("../deep_traffic_generation/data/training_datasets/takeoffs_south_LFPO_07.pkl"),
    features=["track", "groundspeed", "altitude", "timedelta"],
    # scaler=MinMaxScaler(feature_range=(-1,1)),
    scaler=None,
    shape="linear",
    info_params={"features": ["latitude", "longitude"], "index": -1},
)

dataset2 = TrafficDataset.from_file(
    ("../deep_traffic_generation/data/training_datasets/landings_south_LFPO_06.pkl"),
    features=["track", "groundspeed", "altitude", "timedelta"],
    scaler=MinMaxScaler(feature_range=(-1,1)),
    # scaler=None,
    shape="image",
    info_params={"features": ["latitude", "longitude"], "index": -1},
)

In [4]:
traffic1 = Traffic.from_file("../deep_traffic_generation/data/training_datasets/takeoffs_south_LFPO_07.pkl")
traffic2 = Traffic.from_file("../deep_traffic_generation/data/training_datasets/landings_south_LFPO_06.pkl")

data1 = np.stack(
    list([f.flight_id, f.start, f.stop] for f in traffic1)
)
data2 = np.stack(
    list([f.flight_id, f.start, f.stop] for f in traffic2)
)

In [5]:
def calculate_pairs(iter):
    x, y = iter
    delta_t = y[1] - x[1]
    len_x = x[2] - x[1]
    len_y = y[2] - y[1]

    #modified according to sandobox_tcas
    if delta_t < -len_y or (delta_t > len_x):
        return
    
    #make sure that delta_t is smaller than the total duration of the reference (the takeoff)
    elif (delta_t < len_x): 
        return np.array([x[0], y[0], delta_t.total_seconds()])

In [6]:
from multiprocessing import Pool
import itertools
with Pool(processes=os.cpu_count()) as p: 
    pairs = p.map(calculate_pairs, itertools.product(data1,data2))
    p.close()
    p.join()
pairs = np.stack([x for x in pairs if x is not None])

In [30]:
a = []
b = []
for i in range(len(pairs)):
    a.append(dataset1.get_flight(pairs[i,0]))
    b.append(dataset2.get_flight(pairs[i,1]))
    
a = torch.stack(a)
b = torch.stack(b)

In [3]:
filenames1 = next(walk("../deep_traffic_generation/lightning_logs/tcvae/version_0/"+ "checkpoints/"), (None, None, []))[2]
VAE1 = TCVAE.load_from_checkpoint(
    "../deep_traffic_generation/lightning_logs/tcvae/version_0/" + "checkpoints/" + filenames1[0],
    dataset_params=dataset1.parameters,
)
VAE1.eval()

filenames2 = next(walk("../deep_traffic_generation/lightning_logs/tcvae/version_1/"+ "checkpoints/"), (None, None, []))[2]
VAE2 = TCVAE.load_from_checkpoint(
    "../deep_traffic_generation/lightning_logs/tcvae/version_1/" + "checkpoints/" + filename2[0],
    dataset_params=dataset2.parameters,
)
VAE2.eval()

RuntimeError: Error(s) in loading state_dict for TCVAE:
	Missing key(s) in state_dict: "lsr.z_loc.1.weight", "lsr.z_loc.1.bias", "lsr.z_loc.1.running_mean", "lsr.z_loc.1.running_var", "lsr.z_loc.3.weight", "lsr.z_loc.3.bias", "lsr.z_loc.4.weight", "lsr.z_loc.4.bias", "lsr.z_loc.4.running_mean", "lsr.z_loc.4.running_var", "lsr.pseudo_inputs_NN.1.weight", "lsr.pseudo_inputs_NN.1.bias", "lsr.pseudo_inputs_NN.1.running_mean", "lsr.pseudo_inputs_NN.1.running_var", "lsr.pseudo_inputs_NN.3.weight", "lsr.pseudo_inputs_NN.3.bias". 
	Unexpected key(s) in state_dict: "lsr.pseudo_inputs_NN.2.weight", "lsr.pseudo_inputs_NN.2.bias". 

- We selected pairs of trajectories from traffic1 and traffic2

- We make one dataset of takoffs for VAE1 and one dataset of landings for VAe2 based on the pairs (in one dataset, one trajectory may appear several times). Each input tensor is sorted according the pairs we identified.

- We calculate the corresponding latent representations in VAE1 and VAE2 (we may take the means of each posterior or one randomly point). The output tensors are sorted according the pairs we identified.

- We concatenate the two tensors and train the meta VAE. input: linear tensor of size(nb_pairs, 2, latent_dim) or size(nb_pairs, 2*latent_dim). Either we only do TCN VAE or Dense VAE.  

- If everything works well: We are able to generate a new pair of latent representations with the meta VAE. Then we pass each representation in the corresponding VAE to get the reconstructed trajectory. 